In [ ]:
import geopandas
import pandas as pd
import h3
import matplotlib.pyplot as plt

In [ ]:
# Descargar y descomprimir el archivo del Censo y mapa de Arbolado de la Ciudad de Buenos Aires
# arbolado-publico-lineal-2017-2018.zip en el directorio ./DATA

!wget -N -P ./DATA https://cdn.buenosaires.gob.ar/datosabiertos/datasets/atencion-ciudadana/arbolado-publico-lineal/arbolado-publico-lineal-2017-2018.zip
!unzip -o ./DATA/arbolado-publico-lineal-2017-2018.zip -d ./DATA

In [ ]:
def apply_geo_to_h3(row):
    geo_point = row['geometry']
    lat, lng = geo_point.y, geo_point.x  # Extrae latitud y longitud del punto
    resolution = 9  # Define la resolución H3  
    # Aplica la función geo_to_h3 para convertir las coordenadas a H3
    h3_index = h3.geo_to_h3(lat, lng, resolution)
    return h3_index

In [ ]:
# Datos de los arboles geolocalizados
data = geopandas.read_file('./DATA/arbolado-publico-lineal-2017-2018.shp')
# Lookup table que contiene el origen de cada especie
especies = pd.read_csv('./DATA/especies.csv')

In [ ]:
# Visualizacion de los datos (quick & dirty)
data.plot()
plt.show()

In [ ]:
# Agregar el origen de la especie a cada observacion
data = data.merge(especies, how='left', left_on='nombre_cie', right_on='nombre_cie')
# Eliminar registros sin el dato de geometria disponible
delete_index = data[data['geometry'].isna()].index
data.drop(delete_index, inplace=True)
# Calcular el index H3 del poligono al que pertenece cada observacion
data['h3_index'] = data.apply(apply_geo_to_h3, axis=1)

In [ ]:
# Calcular el ratio Autoctonas sobre el total de arboles por poligono H3
ratios_h3 = pd.crosstab(data['h3_index'], data['origen'])
ratios_h3['ratio'] = ratios_h3['Autoctona'] / (ratios_h3['Foranea'] + ratios_h3['Autoctona'])

In [ ]:
ratios_h3.describe(percentiles=[.05, .1, .25, .5, .75, .90, .95, .99])

In [ ]:
ratios_h3['ratio'].hist(bins=30)
plt.show()

In [ ]:
# Poligonos con mayor concentracion de especies autoctonas
# ~5% de los casos

p95_h3 = ratios_h3[ratios_h3['ratio'] >= .22].copy()

In [ ]:
ratios_h3.to_csv('./OUTPUT/ratios_h3.csv')
p95_h3.to_csv('./OUTPUT/p95_h3.csv')